# Customer Churn Prediction

Customer churn or customer attrition is a tendency of clients or customers to abandon a brand and stop being paying clients of a particular business or organization. The percentage of customers discontinuing using a company’s services or products during a specific period is called a customer churn rate. Several bad experiences (or just one) are enough, and a customer may quit. And if a large chunk of unsatisfied customers churn at a time interval, both material losses and damage to reputation would be enormous.

Using machine learning techniques, financial services companies use their customer data to identify behavior patterns of potential churners, classify these at-risk customers, and take appropriate actions to gain their trust and increase their retention rate. The task is to classify customers as churners or non-churners (binary classification) based on the organization’s historical data sources.

To perform the classification task, you can use machine learning classification algorithms like Logistic regression, Naive Bayes Classifier, Tree-based algorithms, Random Forest, etc. To increase efficiency, you can use advanced algorithms like XGBoost, LightGBM, or CatBoost. Use the accuracy metrics to compare the performance of the different models. You can use the Bank Customers Dataset for Churn to practice this project.

https://www.projectpro.io/article/projects-on-machine-learning-applications-in-finance/510#mcetoc_1gla0jmc415

#### Train and Evaluate the Neural Network Model

1. Train (fit) the neural network model. Use the training data and set 100 epochs.

2. Evaluate the model using the test data to determine its loss and accuracy.


## References:

[Keras Sequential model](https://keras.io/api/models/sequential/)

[Keras Dense module](https://keras.io/api/layers/core_layers/dense/)

[Keras evaluate](https://keras.io/api/models/model_training_apis/)

[SKLearn OneHotEncoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html)


In [1]:
# Imports
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

---

## Preprocess the data.

### Step 1: Read the `Churn_Modelling.csv` file from the Resources folder and create a DataFrame.

In [2]:
# Read the HR-Employee-Attrition.csv file from the Resources folder into a Pandas DataFrame
customer_df = pd.read_csv(Path("./Resources/Churn_Modelling.csv"))
customer_df

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [4]:
# data cleaninng: checking number of null
customer_df.isnull().sum()

RowNumber          0
CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

### Step 2: Review the resulting DataFrame. Check the data type associated with each column to identify categorical and non-categorical variables.

> **Hint** Recall that categorical variables have an `object` data type.


In [3]:
# Review the DataFrame
display(customer_df.head())

# Review the data types associated with the columns
display(customer_df.dtypes)
display(customer_df.shape)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


RowNumber            int64
CustomerId           int64
Surname             object
CreditScore          int64
Geography           object
Gender              object
Age                  int64
Tenure               int64
Balance            float64
NumOfProducts        int64
HasCrCard            int64
IsActiveMember       int64
EstimatedSalary    float64
Exited               int64
dtype: object

(10000, 14)

### Step 3: Create a list of categorical variables.

In [5]:
# Create a list of categorical variables 
categorical_variables = list(customer_df.dtypes[customer_df.dtypes == "object"].index)
categorical_variables = ['Geography', 'Gender']

In [6]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Encode categorcal variables using OneHotEncoder
encoded_data = enc.fit_transform(customer_df[categorical_variables])

# Create a DataFrame with the encoded variables
# The column names should match those of the encoded variables
encoded_df = pd.DataFrame(encoded_data,columns = enc.get_feature_names_out(categorical_variables))
encoded_df

,Geography_France,Geography_Germany,Geography_Spain,Gender_Female,Gender_Male
0,1.0,0.0,0.0,1.0,0.0
1,0.0,0.0,1.0,1.0,0.0
2,1.0,0.0,0.0,1.0,0.0
3,1.0,0.0,0.0,1.0,0.0
4,0.0,0.0,1.0,1.0,0.0
...,...,...,...,...,...
9995,1.0,0.0,0.0,0.0,1.0
9996,1.0,0.0,0.0,0.0,1.0
9997,1.0,0.0,0.0,1.0,0.0
9998,0.0,1.0,0.0,0.0,1.0


In [7]:
categorical_variables = encoded_df.columns
categorical_variables





Index(['Geography_France', 'Geography_Germany', 'Geography_Spain',
       'Gender_Female', 'Gender_Male'],
      dtype='object')

In [8]:
# Create a DataFrame with the columnns containing numerical variables from the original dataset
numerical_variables_df = customer_df.drop(columns = ['Surname','Geography', 'Gender'])

# Using the Pandas concat function, combine the DataFrames the contain the encoded categorical data and the numerical data
attrition_df = pd.concat([numerical_variables_df,encoded_df],axis=1)

# Reveiw the DataFrame
attrition_df.head()

,RowNumber,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain,Gender_Female,Gender_Male
0,1,15634602,619,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0,1.0,0.0
1,2,15647311,608,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0,1.0,0.0
2,3,15619304,502,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0,1.0,0.0
3,4,15701354,699,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0,1.0,0.0
4,5,15737888,850,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0,1.0,0.0


In [9]:
# Define the target set y using the Attrition_Yes column
y = attrition_df["Exited"]


# Define features set X by selecting all columns but Attrition_Yes and Attrition_No
X = attrition_df.drop(columns=["Exited"])

# Review the features DataFrameb
display(y.shape)
display(X.shape)

(10000,)

(10000, 15)

### Step 7: Create the training and testing sets using the `train_test_split` function from scikit-learn.

In [12]:
# Split the data into training and testing datasets
# We can use the train_test_split to first make the split on the original dataset. 
# Then, to get the validation set, we can apply the same function to the train set to get the validation set.

# https://towardsdatascience.com/how-to-split-data-into-three-sets-train-validation-and-test-and-why-e50d22d3e54c

# Let's say we want to split the data in 80:10:10 for train:valid:test dataset
train_size=0.8

# In the first step we will split the data in training and remaining dataset
X_train, X_rem, y_train, y_rem = train_test_split(X,y, train_size=0.8)

# Now since we want the valid and test size to be equal (10% each of overall data). 
# we have to define valid_size=0.5 (that is 50% of remaining data)
test_size = 0.5
X_valid, X_test, y_valid, y_test = train_test_split(X_rem,y_rem, test_size=0.5)

print(X_train.shape), print(y_train.shape)
print(X_valid.shape), print(y_valid.shape)
print(X_test.shape), print(y_test.shape)



(8000, 15)
(8000,)
(1000, 15)
(1000,)
(1000, 15)
(1000,)


(None, None)

In [13]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler to the features training dataset
X_scaler = scaler.fit(X_train)

# Fit the scaler to the features training dataset
X_train_scaled = X_scaler.transform(X_train)
X_valid_scaled = X_scaler.transform(X_valid)
X_test_scaled = X_scaler.transform(X_test)

---

## Create a Neural Network Model to Predict Employee Attrition

### Step 1: Use the optimization techniques presented in this lesson to create a deep neural network model with two hidden layers.

In [10]:
# Define the the number of inputs (features) to the model (i.e. 55 = columns of X)
number_input_features = len(X_train.iloc[0])

# Define the number of hidden nodes for the first hidden layer
# Use the mean of the number of input features plus the number of output nurons
# Use the Python floor division (//) to return the quotent
hidden_nodes_layer1 =  (number_input_features + 1) // 2 

# Define the number of hidden nodes for the second hidden layer
# Use the mean of the number of hidden nodes in the first hidden layer plus the number of output nurons
# Use the Python floor division (//) to return the quotent
hidden_nodes_layer2 = (hidden_nodes_layer1 + 1) // 2

# Create the Sequential model instance
nn = Sequential()

# Add the first hidden layer specifying the number of inputs, the number of hidden nodes, and the activation function
nn.add(Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Add the second hidden layer specifying the number of hidden nodes and the activation function
nn.add(Dense(units=hidden_nodes_layer2, activation="relu"))

# Add the output layer to the model specifying the number of output neurons and activation function
# This model will predict a binary output. So, add an output layer with one neuron, and use the `sigmoid` activation function.
nn.add(Dense(units=1, activation="sigmoid"))

# Display the Sequential model summary
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 8)                 128       
                                                                 
 dense_1 (Dense)             (None, 4)                 36        
                                                                 
 dense_2 (Dense)             (None, 1)                 5         
                                                                 
Total params: 169
Trainable params: 169
Non-trainable params: 0
_________________________________________________________________


### Step 2: Compile the model. Use the `binary_crossentropy` loss function, the `adam` optimizer, and the `accuracy` metric.

In [11]:
# Compile the Sequential model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

### Step 3: Train (fit) the neural network model. Use the training data and set 100 epochs.


In [12]:
# Fit the model using 100 epochs and the training data
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
235/235 [==============================] - 1s 1ms/step - loss: 0.5383 - accuracy: 0.7959
Epoch 2/100
235/235 [==============================] - 0s 1ms/step - loss: 0.4670 - accuracy: 0.7963
Epoch 3/100
235/235 [==============================] - 0s 1ms/step - loss: 0.4430 - accuracy: 0.7963
Epoch 4/100
235/235 [==============================] - 0s 1ms/step - loss: 0.4320 - accuracy: 0.7961
Epoch 5/100
235/235 [==============================] - 0s 1ms/step - loss: 0.4265 - accuracy: 0.7964
Epoch 6/100
235/235 [==============================] - 0s 2ms/step - loss: 0.4226 - accuracy: 0.7967
Epoch 7/100
235/235 [==============================] - 0s 2ms/step - loss: 0.4193 - accuracy: 0.8052
Epoch 8/100
235/235 [==============================] - 0s 2ms/step - loss: 0.4167 - accuracy: 0.8172
Epoch 9/100
235/235 [==============================] - 0s 2ms/step - loss: 0.4139 - accuracy: 0.8196
Epoch 10/100
235/235 [==============================] - 0s 2ms/step - loss: 0.4110 - accura

### Step 4: Evaluate the model using the test data to determine its loss and accuracy.

In [13]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

79/79 - 0s - loss: 0.3568 - accuracy: 0.8604 - 225ms/epoch - 3ms/step
Loss: 0.356794536113739, Accuracy: 0.8604000210762024


# Modelling using Logistic Regression with the Original Data

In [ ]:
# Import the modules
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

import warnings
warnings.filterwarnings('ignore')

---

###  Step 1: Fit a logistic regression model by using the training data (`X_train` and `y_train`).

In [ ]:
# Import the LogisticRegression module from SKLearn
from sklearn.linear_model import LogisticRegression

# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
# YOUR CODE HERE!
classifier = LogisticRegression(solver='lbfgs', random_state=1)
classifier
# Fit the model using training data
# YOUR CODE HERE!
classifier.fit(X_train_scaled, y_train)

LogisticRegression(random_state=1)

### Step 2: Save the predictions on the testing data labels by using the testing feature data (`X_test`) and the fitted model.

In [ ]:
# Make a prediction using the testing data
# YOUR CODE HERE!
predictions = classifier.predict(X_test_scaled)

### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [ ]:
# Print the balanced_accuracy score of the model
# YOUR CODE HERE!
from sklearn.metrics import balanced_accuracy_score
# Display the accuracy score for the test dataset.
balanced_accuracy_score(y_test, predictions)

0.59502327689202

In [ ]:
# Generate a confusion matrix for the model
# YOUR CODE HERE!
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, predictions)

array([[755,  28],
       [168,  49]], dtype=int64)

In [ ]:
# Print the classification report for the model
# YOUR CODE HERE!
from sklearn.metrics import classification_report
target_names = ["Exited-0", "Stayed-1"]
print(classification_report(y_test, predictions, target_names=target_names))

              precision    recall  f1-score   support

    Exited-0       0.82      0.96      0.89       783
    Stayed-1       0.64      0.23      0.33       217

    accuracy                           0.80      1000
   macro avg       0.73      0.60      0.61      1000
weighted avg       0.78      0.80      0.77      1000



---

## Predict a Logistic Regression Model with Resampled Training Data

### Step 1: Use the `RandomOverSampler` module from the imbalanced-learn library to resample the data. Be sure to confirm that the labels have an equal number of data points. 

In [ ]:
# Import the RandomOverSampler module form imbalanced-learn
from imblearn.over_sampling import RandomOverSampler

# Instantiate the random oversampler model
# # Assign a random_state parameter of 1 to the model
# YOUR CODE HERE!
random_oversampler = RandomOverSampler(random_state=1)

# Fit the original training data to the random_oversampler model
# YOUR CODE HERE!
X_resampled, y_resampled = random_oversampler.fit_resample(X_train_scaled, y_train)



In [ ]:
# Count the distinct values of the resampled labels data
# YOUR CODE HERE!
y_resampled.value_counts()

0    6367
1    6367
Name: Exited, dtype: int64

### Step 2: Use the `LogisticRegression` classifier and the resampled data to fit the model and make predictions.

In [ ]:
# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
# YOUR CODE HERE!
resampled_classifier = LogisticRegression( random_state=1)
resampled_classifier
# Fit the model using the resampled training data
# YOUR CODE HERE!
resampled_classifier.fit(X_resampled, y_resampled)
# Make a prediction using the testing data
# YOUR CODE HERE!
resampled_predictions = resampled_classifier.predict(X_test_scaled)

### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [ ]:
# Print the balanced_accuracy score of the model 
# YOUR CODE HERE!
balanced_accuracy_score(y_test, resampled_predictions)

0.711799118361966

In [ ]:
# Generate a confusion matrix for the model
# YOUR CODE HERE!
confusion_matrix(y_test, resampled_predictions)

array([[559, 224],
       [ 63, 154]], dtype=int64)

In [ ]:
# Print the classification report for the model
# YOUR CODE HERE!
from imblearn.metrics import classification_report_imbalanced

target_names = ["Exited-0", "Stayed-1"]
print(classification_report_imbalanced(y_test, resampled_predictions, target_names=target_names))

                   pre       rec       spe        f1       geo       iba       sup

   Exited-0       0.90      0.71      0.71      0.80      0.71      0.51       783
   Stayed-1       0.41      0.71      0.71      0.52      0.71      0.51       217

avg / total       0.79      0.71      0.71      0.74      0.71      0.51      1000



---

# Modelling using Random Forest 

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [ ]:
# Create a random forest classifier
rf_model = RandomForestClassifier(n_estimators=500, random_state=78)



In [ ]:
# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)


### Making Predictions Using the Decsion Tree Model

In [ ]:
# Making predictions using the testing data
predictions = rf_model.predict(X_test_scaled)


### Model Evaluation

In [ ]:
# Calculating the confusion matrix
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)

# Calculating the accuracy score
acc_score = accuracy_score(y_test, predictions)



In [ ]:
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
target_names = ["Exited-0", "Stayed-1"]
print(classification_report(y_test, predictions, target_names=target_names))


Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,758,25
Actual 1,128,89


Accuracy Score : 0.847
Classification Report
              precision    recall  f1-score   support

    Exited-0       0.86      0.97      0.91       783
    Stayed-1       0.78      0.41      0.54       217

    accuracy                           0.85      1000
   macro avg       0.82      0.69      0.72      1000
weighted avg       0.84      0.85      0.83      1000



## Feature Importance

In [ ]:
# Random Forests in sklearn will automatically calculate feature importance
importances = rf_model.feature_importances_



In [ ]:
# We can sort the features by their importance
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)



[(0.20490101667565316, 'Age'),
 (0.1186610360552868, 'NumOfProducts'),
 (0.10499725815923944, 'Balance'),
 (0.10098306344264969, 'EstimatedSalary'),
 (0.10028852853784988, 'CustomerId'),
 (0.100143825403658, 'CreditScore'),
 (0.09913368126434659, 'RowNumber'),
 (0.05999950210564481, 'Tenure'),
 (0.03966840855940236, 'IsActiveMember'),
 (0.021179640239053377, 'Geography_Germany'),
 (0.014946724436429703, 'HasCrCard'),
 (0.009782349014218445, 'Geography_France'),
 (0.008638956174708678, 'Geography_Spain'),
 (0.008504718776518073, 'Gender_Female'),
 (0.008171291155340965, 'Gender_Male')]

---

# Modelling using K-Nearest Neighbors   

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

# Instantiate the model with k = 3 neighbors
model = KNeighborsClassifier(n_neighbors=3)

In [ ]:
# Train the model
model.fit(X_train_scaled, y_train)

KNeighborsClassifier(n_neighbors=3)

In [ ]:
# Create predictions
y_pred = model.predict(X_test_scaled)

# Review the predictions
y_pred

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0,
       0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,

In [ ]:
# Print confusion matrix
confusion_matrix(y_pred,y_test)

array([[732, 144],
       [ 51,  73]], dtype=int64)

In [ ]:
# Print classification report
print(classification_report(y_pred,y_test, target_names=target_names))

              precision    recall  f1-score   support

    Exited-0       0.93      0.84      0.88       876
    Stayed-1       0.34      0.59      0.43       124

    accuracy                           0.81      1000
   macro avg       0.64      0.71      0.66      1000
weighted avg       0.86      0.81      0.83      1000



---